In [ ]:
# env
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

In [2]:
# LLM
from langchain_anthropic import ChatAnthropic
llm = ChatAnthropic(model="claude-3-5-sonnet-latest")

In [ ]:
import mermaid as md
from mermaid.graph import Graph

fhir_diagram = Graph('FHIR-Flow', """
graph TD
    D[D] --> FHIR
    FHIR --> Enc[Enc.]
    Enc --> Obs[Obs]
    Enc --> Proc[Proc]
    Enc --> Meds[Meds]
    Obs --> LOINC
    Proc --> CPTR
    Meds --> RxNorm
    LOINC --> Evaluator
    Evaluator -->|Y| Out[Out]
    Evaluator -->|N| Reject[Reject: feedback to generator]

    subgraph LOINC_Checks
        note1["1. Detect code use"]
        note2["2. Identify code type"]
        note3["3. RAG: code exists"]
        note4["4. RAG: code is appropriate?"]
    end
""")

render = md.Mermaid(fhir_diagram)
render  # This will render in a Jupyter notebook environment


In [ ]:
from graphviz import Digraph

dot = Digraph(comment='FHIR Flow Diagram')
dot.attr(rankdir='TB')  # Top to Bottom direction

# Add nodes
dot.node('D', 'D')
dot.node('FHIR', 'FHIR')
dot.node('Enc', 'Enc.')
dot.node('Obs', 'Obs')
dot.node('Proc', 'Proc')
dot.node('Meds', 'Meds')
dot.node('LOINC', 'LOINC')
dot.node('CPTR', 'CPTR')
dot.node('RxNorm', 'RxNorm')
dot.node('Evaluator', 'Evaluator')
dot.node('Out', 'Out')
dot.node('Reject', 'Reject: feedback to generator')

# Add edges
dot.edge('D', 'FHIR')
dot.edge('FHIR', 'Enc')
dot.edge('Enc', 'Obs')
dot.edge('Enc', 'Proc')
dot.edge('Enc', 'Meds')
dot.edge('Obs', 'LOINC')
dot.edge('Proc', 'CPTR')
dot.edge('Meds', 'RxNorm')
dot.edge('LOINC', 'Evaluator')
dot.edge('Evaluator', 'Out', 'Y')
dot.edge('Evaluator', 'Reject', 'N')

# Add LOINC evaluation steps as a subgraph
with dot.subgraph(name='cluster_0') as c:
    c.attr(label='LOINC Checks')
    c.node('step1', '1. Detect code use')
    c.node('step2', '2. Identify code type')
    c.node('step3', '3. RAG: code exists')
    c.node('step4', '4. RAG: code is appropriate?')

# Save the diagram
dot.render('fhir_flow', format='png', cleanup=True)


In [3]:
from diagrams import Diagram, Cluster
from diagrams.custom import Custom
from diagrams.programming.language import Python  # We'll use this as a generic node

# You'll need to create this in a function since Diagram is a context manager
def create_fhir_diagram():
    with Diagram("FHIR Flow", filename="fhir_flow_pydiagrams", show=True):
        # Create main nodes
        d = Python("D")
        fhir = Python("FHIR")
        enc = Python("Enc.")
        
        # Create resource nodes
        obs = Python("Obs")
        proc = Python("Proc")
        meds = Python("Meds")
        
        # Create terminology nodes
        loinc = Python("LOINC")
        cptr = Python("CPTR")
        rxnorm = Python("RxNorm")
        
        # Create evaluation nodes
        evaluator = Python("Evaluator")
        out = Python("Out")
        reject = Python("Reject: feedback to generator")
        
        # Create main flow
        d >> fhir >> enc
        enc >> obs >> loinc
        enc >> proc >> cptr
        enc >> meds >> rxnorm
        
        # Create evaluation flow
        loinc >> evaluator
        evaluator >> out
        evaluator >> reject
        
        # Create LOINC checks cluster
        with Cluster("LOINC Checks"):
            checks = [
                Python("1. Detect code use"),
                Python("2. Identify code type"),
                Python("3. RAG: code exists"),
                Python("4. RAG: code is appropriate?")
            ]




In [ ]:
create_fhir_diagram()

In [24]:
from diagrams import Diagram, Cluster, Edge
from diagrams.programming.flowchart import Action, Decision, Database, InputOutput
from diagrams.programming.language import Python  # Better for representing JSON/data objects

def create_fhir_diagram_v2():
    with Diagram("FHIR Flow", filename='fhir_flow_pydiagrams_2', show=False):
        # Create main nodes
        d = InputOutput("Dialog \n[+ Feedback]")  # Input node
        fhir = Python("FHIR")  # Python Generic node
        enc = Action("Enc.")  # Action/Process node
        rag = Action("RAG")  # Action/Process node
        
        # Create resource nodes
        obs = Action("Obs")
        proc = Action("Proc")
        meds = Action("Meds")
        
        # Create terminology nodes
        loinc = Database("LOINC")
        cptr = Database("CPTR")
        rxnorm = Database("RxNorm")
        vectorstore = Database("Vector Stores")
        
        # Create evaluation nodes
        evaluator = Decision("Evaluator:\nAre codes valid\nand appropriate?")  # Decision point
        out = InputOutput("Out")  # Output node
        reject = InputOutput("Reject: feedback to generator")  # Rejection output
        
        # Create main flow
        d >> fhir >> enc
        enc >> obs >> loinc
        enc >> proc >> cptr
        enc >> meds >> rxnorm
        
        # Create evaluation flow
        loinc >> evaluator
        cptr >> evaluator
        rxnorm >> evaluator
        
        evaluator >> Edge(label="Y") >> out
        evaluator >> Edge(label="N") >> reject

        # Loop back to the beginning
        reject >> d

        # Evaluator RAG loop
        evaluator >> Edge(label="RAG") >> rag
        rag >> vectorstore
        vectorstore >> rag
        rag  >> evaluator

        
        # Create LOINC checks cluster
        with Cluster("LOINC Checks"):
            checks = [
                Action("1. Detect code use"),
                Action("2. Identify code type"),
                Decision("3. RAG: code exists"),
                Decision("4. RAG: code is appropriate?")
            ]

if __name__ == "__main__":
    create_fhir_diagram_v2()
